In [1]:
from utils.models_dot import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

## Load

In [2]:
run = 1

In [3]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
        "train_data/train.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=8,
    shuffle=True,
)
val_dataloader = DataLoader(
    RatingsDataset(
        "train_data/validation.csv",
        "user_id",
        "movie_id",
        "rating",
    ),
    batch_size=1024,
    num_workers=4,
    shuffle=True,
)

In [4]:
import torch
import torch.nn as nn
import torch.optim as opt

In [8]:
torch.manual_seed(2)

In [9]:
model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

model

NeuFM(
  (gmf): GMF(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Linear(in_features=8, out_features=1, bias=False)
  )
)

## Train

In [10]:
criterion = nn.MSELoss().cuda()
optimizer = opt.Adam(model.parameters(), lr=0.0005)

prev_val_loss = math.inf
for epoch in tqdm(range(50)):
    n_batches = len(train_dataloader)
    avg_loss = 0
    val_loss = 0
    
    # Train step
    for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        optimizer.zero_grad()
        y_hat = model(vus, vis)
        
        loss = criterion(y_hat, rs)
        loss.backward()
        optimizer.step()
        
        avg_loss += math.sqrt(float(loss.detach().cpu()))
        
    # Val step
    with torch.no_grad():
        for val_vus, val_vis, val_rs in val_dataloader:
            val_vus = val_vus.cuda()
            val_vis = val_vis.cuda()
            val_rs = val_rs.cuda()

            val_pred = model(val_vus, val_vis)
            val_loss += math.sqrt(float(criterion(val_pred, val_rs).detach().cpu()))
                
    val_loss /= len(val_dataloader)
    print(f"epoch: {epoch}, train_loss: {avg_loss/n_batches}, val_loss: {val_loss}")
    
    if val_loss < prev_val_loss:
        prev_val_loss = val_loss
#         torch.save(
#             model.state_dict(), 
#             f"/home/nubol23/Documents/NCF_dot_weights/run_{run}/{epoch}-{val_loss}.pt"
#         )

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0, train_loss: 2.956326577171081, val_loss: 1.1161064901715423
epoch: 1, train_loss: 1.0035022194643075, val_loss: 0.9561162859263271
epoch: 2, train_loss: 0.9288072963526742, val_loss: 0.9236659219532888
epoch: 3, train_loss: 0.9073647238664276, val_loss: 0.9109747109041914
epoch: 4, train_loss: 0.896532083944644, val_loss: 0.9036692045590853
epoch: 5, train_loss: 0.8889849994819201, val_loss: 0.8982821873242268
epoch: 6, train_loss: 0.8826675148167501, val_loss: 0.8942432890079535
epoch: 7, train_loss: 0.8770832853939032, val_loss: 0.8903219043198978
epoch: 8, train_loss: 0.8719390695069579, val_loss: 0.8876649836486495
epoch: 9, train_loss: 0.8673854669597011, val_loss: 0.885112780472094
epoch: 10, train_loss: 0.8634863441331718, val_loss: 0.8832531377777159
epoch: 11, train_loss: 0.8596058190810888, val_loss: 0.8814296882799986
epoch: 12, train_loss: 0.856179935249112, val_loss: 0.8798964143435043
epoch: 13, train_loss: 0.8528883582675542, val_loss: 0.8786006573976052
epoch:

KeyboardInterrupt: 

## Predict

In [ ]:
trained_weights = ""

In [ ]:
trained_model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

trained_model.load_state_dict(
    torch.load(
        f"/home/nubol23/Documents/NCF_dot_weights/run_{run}/{trained_weights}"
    )
)
trained_model.cuda().eval()

In [ ]:
test_dataloader = DataLoader(
    RatingsDataset(
        "train_data/test.csv",
        "user_id",
        "movie_id",
    ), 
    batch_size=1024,
    num_workers=12
)

In [ ]:
test_predictions = []

for vus, vis in test_dataloader:
    vus = vus.cuda()
    vis = vis.cuda()

    pred = torch.clip(trained_model(vus, vis), 1, 5).cpu().ravel().tolist()
    test_predictions += pred

In [ ]:
import pandas as pd

test_csv = pd.read_csv("../../data/test_data.csv")

In [ ]:
out_df = pd.DataFrame.from_dict(
    {
        "id": list(test_csv["id"]),
        "rating": test_predictions
    }
)

out_df.head()

In [ ]:
# out_df.to_csv(f"outputs_csv/neumf_5.csv", index=False)